# 08 - Corriere dello Sport

In [1]:
from requests import get
from pandas import read_csv
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as BS
from time import sleep
from unidecode import unidecode

## Articoli di qualunque data

In [5]:
df = read_csv('tables/nome_nomechiave.csv', encoding='ansi')
df['nome_chiave_link'] = [x.replace(' ', '+').replace('-','+').lower() for x in df['nome_chiave']]
s1 = 'http://www.corrieredellosport.it/search?key=('
df['urls_all'] = [s1+x+')' for x in df.nome_chiave_link]
df['corriere_all'] = np.nan
df = df[['nome', 'urls_all', 'corriere_all']]
df.head(10)

,nome,urls_all,corriere_all
0,Kylian Mbappé,http://www.corrieredellosport.it/search?key=(m...,NaN
1,Neymar,http://www.corrieredellosport.it/search?key=(n...,NaN
2,Leo Messi,http://www.corrieredellosport.it/search?key=(m...,NaN
3,Mohamed Salah,http://www.corrieredellosport.it/search?key=(s...,NaN
4,Harry Kane,http://www.corrieredellosport.it/search?key=(h...,NaN
5,Antoine Griezmann,http://www.corrieredellosport.it/search?key=(g...,NaN
6,Kevin De Bruyne,http://www.corrieredellosport.it/search?key=(d...,NaN
7,Philippe Coutinho,http://www.corrieredellosport.it/search?key=(c...,NaN
8,Eden Hazard,http://www.corrieredellosport.it/search?key=(h...,NaN
9,Paulo Dybala,http://www.corrieredellosport.it/search?key=(d...,NaN


In [22]:
for i in range(300):
    url = df.urls_all[i]
    
    # solito trucco per riempire man mano la colonna
    # così se da errore non perdo nulla
    # e posso proseguire riavviando la cella:
    
    if isinstance(df.corriere_all[i], float):
        
        # uso espressione regex (pacchetto "re")
        # per non passare da Beautiful Soup
        
        l = re.findall('>+(\d+)\s+articoli</a>', get(url).text)
        
        # if lista: <--- questa espressione dà True
        # se la lista ha almeno un elemento dentro di essa.
        # serve per evitare errori banali tipo lista[0]
        # ma la lista è vuota quindi [0] non esiste --> errore
        
        if l:
            df.loc[i, 'corriere_all'] = l[0]

In [26]:
df.head()

,nome,urls_all,corriere_all
0,Kylian Mbappé,http://www.corrieredellosport.it/search?key=(m...,658
1,Neymar,http://www.corrieredellosport.it/search?key=(n...,2300
2,Leo Messi,http://www.corrieredellosport.it/search?key=(m...,5413
3,Mohamed Salah,http://www.corrieredellosport.it/search?key=(s...,2155
4,Harry Kane,http://www.corrieredellosport.it/search?key=(h...,231


## Articoli del 2018

In [79]:
df['corriere_18'] = np.nan

Funzione che prende in input una pagina e restituisce il numero di volte che appare 2018 nei link agli articoli di quella pagina:

In [80]:
def sum18(url):
    p = BS(get(url).text,'lxml')
    y = p.select('div.main-section__main article.article div.article__content a')
    return sum(['2018' in x['href'] for x in y])

In [ ]:
for i in range(300):
    s = np.nan
    
    # solito trucco per riempire man mano la colonna
    # così se da errore non perdo nulla
    # e posso proseguire riavviando la cella:
    
    if isinstance(df.corriere_18[i], float):
        
        url = df.urls_all[i]+'%20AND%20(type%3Aarticolo)'
        s = sum18(url)
        N = int(p.select('span.page')[0].text.split(' ')[-1])
        
        # ciclo for nelle pagine alla ricerca delle date con 2018 nei link
        # in ogni pagina ci sono 10 link i quali possono contenere o meno 2018
        # se nessun link contiene 2018 vuol dire che posso uscire dal ciclo
        # infatti pare che siano ordinati cronologicamente (grazie a dio)
        # sommo ad ogni passo il numero di volte che il link contiene 2018
        # cumulo le somme, e se la somma è 0 allora break
        
        for j in range(2,N+1):
            
            url2 = url+'&p='+str(j)
            s2 = sum18(url2) 
            s += s2
            
            if s2 == 0:
                break
        
        # salvo come stringa per assicurarmi che non venga visto come numeric
        
        df.loc[i, 'corriere_18'] = str(s)

In [84]:
df.head(5)

,nome,urls_all,corriere_all,corriere_18
0,Kylian Mbappé,http://www.corrieredellosport.it/search?key=(m...,658,305
1,Neymar,http://www.corrieredellosport.it/search?key=(n...,2300,569
2,Leo Messi,http://www.corrieredellosport.it/search?key=(m...,5413,1385
3,Mohamed Salah,http://www.corrieredellosport.it/search?key=(s...,2155,404
4,Harry Kane,http://www.corrieredellosport.it/search?key=(h...,231,108


In [85]:
df[['corriere_all', 'corriere_18']].to_csv('columns/corriere_dello_sport.csv')